In [1]:
import numpy as np
import cv2
import time
import os
from collections import defaultdict
from PIL import Image
import configparser
import interpolation as inter

%load_ext autoreload
%autoreload 2


ROI_RECTANGLE = 0
ROI_POLYGON = 1

EDSR = -1
INTER_NEAREST = cv2.INTER_NEAREST
INTER_LINEAR = cv2.INTER_LINEAR
INTER_CUBIC = cv2.INTER_CUBIC
INTER_AREA = cv2.INTER_AREA
INTER_LANCZOS4 = cv2.INTER_LANCZOS4
INTER_LINEAR_EXACT = cv2.INTER_LINEAR_EXACT
INTER_NEAREST_EXACT = cv2.INTER_NEAREST_EXACT
INTER_MAX = cv2.INTER_MAX
WARP_FILL_OUTLIERS = cv2.WARP_FILL_OUTLIERS
WARP_INVERSE_MAP = cv2.WARP_INVERSE_MAP
WARP_RELATIVE_MAP = cv2.WARP_RELATIVE_MAP

drawing = False
points = []

In [6]:

# 용량이 큰 이미지 분할해서 upscaling 할 수 있도록 구현
# 분할된 부분이 겹치도록 - 겹친 후에 블렌딩으로 자연스럽게

PIXELS_LIMIT = 281000
OVERLAP_HALF_LENGTH = 20

_upscaled_fraction_num = 0

def _upscale_img(img, scaler):
    """
    img: 이미지 ndarray
    scaler: 배율(2 or 3 or 4)
    """
    # if scaler not in [2, 3, 4]:
    #     print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
    #     return None
    # if not cv2.cuda.getCudaEnabledDeviceCount():
    #     print("No CUDA-enabled GPU found.")
    #     return None

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    try:
        sr.readModel(f'models/EDSR_x{scaler}.pb')
    except Exception as e:
        print(f"Error reading model: {e}")
        return None

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    try:
        result = sr.upsample(img)
    except Exception as e:
        print(f"Error during upscaling: {e}")
        return None
    global _upscaled_fraction_num
    _upscaled_fraction_num += 1
    return result

def _upscale_large_img_helper(img, scaler):
    if scaler not in [2, 3, 4]:
        print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
        return None
    if not cv2.cuda.getCudaEnabledDeviceCount():
        print("No CUDA-enabled GPU found.")
        return None
    
    h, w, c = img.shape
    if h * w < PIXELS_LIMIT:
        return _upscale_img(img, scaler)
    
    if w < h:
        upper = np.zeros((scaler*h, scaler*w, c))
        below = np.zeros((scaler*h, scaler*w, c))

        upper[0:scaler*(h//2 + OVERLAP_HALF_LENGTH),:] = _upscale_large_img_helper(img[0:(h//2 + OVERLAP_HALF_LENGTH),:], scaler=scaler)
        below[scaler*(h//2 - OVERLAP_HALF_LENGTH):scaler*h,:] = _upscale_large_img_helper(img[(h//2 - OVERLAP_HALF_LENGTH):h,:], scaler=scaler)

        # h//2 - OVERLAP_HALF_LENGTH ~ h//2 + OVERLAP_HALF_LENGTH
        alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * h).reshape(-1, 1)
        start = scaler * (h//2 - OVERLAP_HALF_LENGTH)
        end = scaler * (h//2 + OVERLAP_HALF_LENGTH)
        alpha = np.clip((alpha - start) / (end - start), 0, 1)

        alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)

        upper_f = upper.astype(np.float32)
        below_f = below.astype(np.float32)

        blended = upper_f * (1-alpha_3ch) + below_f * alpha_3ch
        blended = np.clip(blended, 0, 255).astype(np.uint8)
        return blended
    
    else:
        left = np.zeros((scaler*h, scaler*w, c))
        right = np.zeros((scaler*h, scaler*w, c))
        
        left[:,0:scaler*(w//2 + OVERLAP_HALF_LENGTH)] = _upscale_large_img_helper(img[:,0:(w//2 + OVERLAP_HALF_LENGTH)], scaler=scaler)
        right[:,scaler*(w//2 - OVERLAP_HALF_LENGTH):scaler*w] = _upscale_large_img_helper(img[:,(w//2 - OVERLAP_HALF_LENGTH):w], scaler=scaler)

        # w//2 - OVERLAP_HALF_LENGTH ~ w//2 + OVERLAP_HALF_LENGTH
        alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * w).reshape(1, -1)
        start = scaler * (w//2 - OVERLAP_HALF_LENGTH)
        end = scaler * (w//2 + OVERLAP_HALF_LENGTH)
        alpha = np.clip((alpha - start) / (end - start), 0, 1)
        
        alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)
        
        left_f = left.astype(np.float32)
        right_f = right.astype(np.float32)

        blended = left_f * (1-alpha_3ch) + right_f * alpha_3ch
        blended = np.clip(blended, 0, 255).astype(np.uint8)
        return blended

def upscale_large_img(img, scaler):
    """
    img: 이미지 ndarray
    scaler: 배율(2 or 3 or 4)
    """
    global _upscaled_fraction_num
    _upscaled_fraction_num = 0

    t1 = time.time()
    result = _upscale_large_img_helper(img, scaler=scaler)
    t2 = time.time()
    if _upscaled_fraction_num > 1:
        print(f'upscaled after being divided into {_upscaled_fraction_num} fragments.')
    else:
        print(f'upscaled without fraction')
    print(f'{t2-t1} sec taken')
    _upscaled_fraction_num = 0
    return result



In [ ]:
img1 = cv2.imread('Lenna_(test_image).png')
scaler = 4

if scaler not in [2, 3, 4]:
    print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")

if not cv2.cuda.getCudaEnabledDeviceCount():
    print("No CUDA-enabled GPU found.")

sr = cv2.dnn_superres.DnnSuperResImpl_create()
try:
    sr.readModel(f'models/EDSR_x{scaler}.pb')
except Exception as e:
    print(f"Error reading model: {e}")

# gpu acceleration
sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

sr.setModel('edsr', scaler)

try:
    result = sr.upsample(img1)
except Exception as e:
    print(f"Error during upscaling: {e}")


In [ ]:
def upscale_single_img(img, scaler):

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel(f'models/EDSR_x{scaler}.pb')

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    result = sr.upsample(img)

    return result

def upscale_mult_img(scaler, *args):

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel(f'models/EDSR_x{scaler}.pb')

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    result = []
    for img in args:
        res = sr.upsample(img)
        result.append(res)
    return result

_img = cv2.imread('Lenna_(test_image).png')
img2 = cv2.rotate(_img, cv2.ROTATE_90_CLOCKWISE)

t1 = time.time()
upscaleds_mult = upscale_mult_img(4, _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2, 
                                  _img, img2, _img, img2, _img, img2, _img, img2, _img, img2) # 100
t2 = time.time()


upscaleds_sing = []
for _ in range(50):
    up1 = upscale_single_img(_img, 4)
    up2 = upscale_single_img(img2, 4)
    upscaleds_sing.append(up1)
    upscaleds_sing.append(up2)
t3 = time.time()

print(len(upscaleds_mult))
print(len(upscaleds_sing))

print(f'{t2-t1}, {t3-t2} -> {(t2-t1) / (t3-t2)}')


# for upscaled in upscaleds:
#     cv2.imshow('test', upscaled)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


"""
100
100
197.584814786911, 369.85457468032837 -> 0.5342229846898255
"""


100
100
197.584814786911, 369.85457468032837 -> 0.5342229846898255


In [ ]:
"""
안1 함수 안에 함수로 모델 공유하도록
안2 모델을 파라미터로 넘기도록
"""

# TODO: np 대신 cp 적용 -> 메모리 해제 시간 고려 효율



In [22]:
PIXELS_LIMIT = 281000
OVERLAP_HALF_LENGTH = 20

def upsample_large_img(img, scaler):
    upsampled_fraction_num = 0
    
    def upsample_img(_img):
        nonlocal upsampled_fraction_num
        h, w, c = _img.shape
        if h * w < PIXELS_LIMIT:
            upsampled_fraction_num += 1
            return sr.upsample(_img)
        
        if w < h:
            upper = np.zeros((scaler*h, scaler*w, c))
            below = np.zeros((scaler*h, scaler*w, c))

            upper[0:scaler*(h//2 + OVERLAP_HALF_LENGTH),:] = upsample_img(_img[0:(h//2 + OVERLAP_HALF_LENGTH),:])
            below[scaler*(h//2 - OVERLAP_HALF_LENGTH):scaler*h,:] = upsample_img(_img[(h//2 - OVERLAP_HALF_LENGTH):h,:])

            # h//2 - OVERLAP_HALF_LENGTH ~ h//2 + OVERLAP_HALF_LENGTH
            alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * h).reshape(-1, 1)
            start = scaler * (h//2 - OVERLAP_HALF_LENGTH)
            end = scaler * (h//2 + OVERLAP_HALF_LENGTH)
            alpha = np.clip((alpha - start) / (end - start), 0, 1)

            alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)

            upper_f = upper.astype(np.float32)
            below_f = below.astype(np.float32)

            blended = upper_f * (1-alpha_3ch) + below_f * alpha_3ch
            blended = np.clip(blended, 0, 255).astype(np.uint8)
            return blended
        else:
            left = np.zeros((scaler*h, scaler*w, c))
            right = np.zeros((scaler*h, scaler*w, c))
            
            left[:,0:scaler*(w//2 + OVERLAP_HALF_LENGTH)] = upsample_img(_img[:,0:(w//2 + OVERLAP_HALF_LENGTH)])
            right[:,scaler*(w//2 - OVERLAP_HALF_LENGTH):scaler*w] = upsample_img(_img[:,(w//2 - OVERLAP_HALF_LENGTH):w])

            # w//2 - OVERLAP_HALF_LENGTH ~ w//2 + OVERLAP_HALF_LENGTH
            alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * w).reshape(1, -1)
            start = scaler * (w//2 - OVERLAP_HALF_LENGTH)
            end = scaler * (w//2 + OVERLAP_HALF_LENGTH)
            alpha = np.clip((alpha - start) / (end - start), 0, 1)
            
            alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)
            
            left_f = left.astype(np.float32)
            right_f = right.astype(np.float32)

            blended = left_f * (1-alpha_3ch) + right_f * alpha_3ch
            blended = np.clip(blended, 0, 255).astype(np.uint8)
            return blended
        
    if scaler not in [2, 3, 4]:
        print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
        return None
    if not cv2.cuda.getCudaEnabledDeviceCount():
        print("No CUDA-enabled GPU found.")
        return None

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    try:
        sr.readModel(f'models/EDSR_x{scaler}.pb')
    except Exception as e:
        print(f"Error reading model: {e}")
        return None

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    t1 = time.time()
    result = upsample_img(img)
    t2 = time.time()

    if upsampled_fraction_num > 1:
        print(f'upscaled after being divided into {upsampled_fraction_num} fragments.')
    else:
        print(f'upscaled without fraction')
    print(f'{t2-t1} sec taken')

    return result




In [25]:
_img = cv2.imread('sample-images-png/1920x1080.png')

# upsampled1 = upscale_large_img(img, 4)
upsampled2 = upsample_large_img(_img, 4)




upscaled after being divided into 16 fragments.
25.06098985671997 sec taken


In [26]:
"""
upsampled1
upscaled after being divided into 16 fragments.
40.26908230781555 sec taken

upscaled after being divided into 16 fragments.
39.244662284851074 sec taken

"""



"""
upsampled2
upscaled after being divided into 16 fragments.
25.543002367019653 sec taken

upscaled after being divided into 16 fragments.
25.06098985671997 sec taken

"""

'\nupsampled2\nupscaled after being divided into 16 fragments.\n25.543002367019653 sec taken\n\nupscaled after being divided into 16 fragments.\n25.06098985671997 sec taken\n\n'

In [24]:
# cv2.imshow('test', upsampled1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imshow('test', upsampled2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
def foo():
    foovar = 234
    def bar():
        nonlocal foovar
        foovar += 1
        print(foovar)
    bar()

foo()

235


In [3]:
import mrs3 as mr
import cv2

img = cv2.imread('sample-images-png/1920x1080.png')

upscaled = mr.upscale_large_img(img, 4)

cv2.imshow('upscaled', upscaled)
cv2.waitKey(0)
cv2.destroyAllWindows()


upscaled after being divided into 16 fragments.
25.34834623336792 sec taken qqqqq


In [3]:
def _upscale_img_test(img, scaler):
    """
    img: 이미지 ndarray
    scaler: 배율(2 or 3 or 4)
    """
    # if scaler not in [2, 3, 4]:
    #     print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
    #     return None
    # if not cv2.cuda.getCudaEnabledDeviceCount():
    #     print("No CUDA-enabled GPU found.")
    #     return None

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    try:
        sr.readModel(f'models/EDSR_x{scaler}.pb')
    except Exception as e:
        print(f"Error reading model: {e}")
        return None

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    try:
        result = sr.upsample(img)
    except Exception as e:
        print(f"Error during upscaling: {e}")
        return None
    return result

img = cv2.imread('Lenna_(test_image).png')
upscaled = _upscale_img_test(img, 4)
cv2.imshow('test', upscaled)
cv2.waitKey(0)
cv2.destroyAllWindows()